In [3]:
from utils.db import ToricCY
from utils import format
db = ToricCY()

In [4]:
invols = db.find_invols({'H11': 3})

In [107]:
from utils.calc import get_srideal
# ex = invols[65]
ex = invols[126]
rescws = format.mat(ex['RESCWS'])
itensxd = format.mat(ex['ITENSXD'])
nverts = format.mat(ex['NVERTS'])
dresverts = format.mat(ex['DRESVERTS'])
k, n = dresverts.shape
triang = format.mat(ex['TRIANG'])
srideal = get_srideal(triang)
invol = format.invol(ex['INVOL'], k)

In [108]:
import numpy as np
from sage.geometry.polyhedron.constructor import Polyhedron
Delta = Polyhedron(vertices=nverts)
npoints = np.array(Delta.integral_points())

In [109]:
P_expon = (npoints @ dresverts.T) + 1

In [110]:
# from sage.rings.integer_ring import ZZ
# from sage.matrix.all import matrix
# m = matrix(ZZ, P_expon - P_expon[:, invol.array_form]).row_space().matrix()

In [111]:
# P_weights = np.unique(P_expon @ rescws, axis=0)

In [112]:
from sage.rings.rational_field import QQ
R = QQ[tuple(f'x_{i}' for i in range(k))]
X = np.array(R.gens())
X_invol = X[invol.array_form]
X_sub = dict(zip(X, X_invol))

In [113]:
srideal = [R.ideal(*np.array(X)[s]) for s in srideal]

In [130]:
a = P_monoms.sum().reduce(fixed_loci[0])

In [222]:
def is_smooth(P_monoms, fixed_loci, srideal):
    J_P_monoms = np.array([[x.derivative(y) for y in X] for x in P_monoms])
    trans_ideal = R.ideal(*P_monoms) + R.ideal(*J_P_monoms.flatten()) + sum(fixed_loci)
    for s in srideal:
        if trans_ideal.quotient(s).dimension() < 0:
            return True
    return False

In [218]:
a.dimension()

3

[-1, 3, 3]

In [221]:
ex['SMOOTH']

True

In [209]:
srideal[0] + srideal[1]

Ideal (x_0, x_5, x_2, x_3) of Multivariate Polynomial Ring in x_0, x_1, x_2, x_3, x_4, x_5, x_6 over Rational Field

In [198]:
q.groebner_basis(), srideal[0].groebner_basis()

([x_0, x_5, x_6], [x_0, x_5])

In [196]:
q > srideal[0]

True

In [192]:
q = a.minimal_associated_primes()[0]
q, srideal

(Ideal (x_6, x_5, x_0) of Multivariate Polynomial Ring in x_0, x_1, x_2, x_3, x_4, x_5, x_6 over Rational Field,
 [Ideal (x_0, x_5) of Multivariate Polynomial Ring in x_0, x_1, x_2, x_3, x_4, x_5, x_6 over Rational Field,
  Ideal (x_2, x_3) of Multivariate Polynomial Ring in x_0, x_1, x_2, x_3, x_4, x_5, x_6 over Rational Field,
  Ideal (x_1, x_4, x_6) of Multivariate Polynomial Ring in x_0, x_1, x_2, x_3, x_4, x_5, x_6 over Rational Field])

In [179]:
q.intersection(srideal[0])

Ideal (x_5, x_0) of Multivariate Polynomial Ring in x_0, x_1, x_2, x_3, x_4, x_5, x_6 over Rational Field

In [158]:
a.minimal_associated_primes()

[Ideal (x_6, x_5, x_0) of Multivariate Polynomial Ring in x_0, x_1, x_2, x_3, x_4, x_5, x_6 over Rational Field,
 Ideal (x_6, x_4, x_1) of Multivariate Polynomial Ring in x_0, x_1, x_2, x_3, x_4, x_5, x_6 over Rational Field,
 Ideal (x_6, x_3, x_2) of Multivariate Polynomial Ring in x_0, x_1, x_2, x_3, x_4, x_5, x_6 over Rational Field]

In [155]:
srideal

[Ideal (x_0, x_5) of Multivariate Polynomial Ring in x_0, x_1, x_2, x_3, x_4, x_5, x_6 over Rational Field,
 Ideal (x_2, x_3) of Multivariate Polynomial Ring in x_0, x_1, x_2, x_3, x_4, x_5, x_6 over Rational Field,
 Ideal (x_1, x_4, x_6) of Multivariate Polynomial Ring in x_0, x_1, x_2, x_3, x_4, x_5, x_6 over Rational Field]

### O-planes

In [159]:
P_monoms = np.power(X, P_expon).prod(axis=1)
P_diff = np.vectorize(lambda x: x - x.subs(X_sub))(P_monoms)

In [115]:
R.ideal(*[x.reduce(srideal) for x in P_diff]).minimal_associated_primes()

TypeError: no canonical coercion from Monoid of ideals of Multivariate Polynomial Ring in x_0, x_1, x_2, x_3, x_4, x_5, x_6 over Rational Field to Multivariate Polynomial Ring in x_0, x_1, x_2, x_3, x_4, x_5, x_6 over Rational Field

In [116]:
P_diff[1] - P_diff[1].reduce(srideal[0])

x_0^4*x_1*x_3^6*x_4^5 - x_0^4*x_2*x_3^5*x_4^6

In [117]:
(R.ideal(*P_diff) + srideal[3]).minimal_associated_primes()

IndexError: list index out of range

In [118]:
fixed_loci = R.ideal(*P_diff).minimal_associated_primes()

In [119]:
def get_odim(I):
    codim = I.dimension()
    dim = I.ring().krull_dimension() - codim
    return 3 + 2 * (3 - dim)

In [225]:
a = P_monoms[0]

In [228]:
a.parent()

Multivariate Polynomial Ring in x_0, x_1, x_2, x_3, x_4, x_5, x_6 over Rational Field

In [121]:
srideal

[Ideal (x_0, x_5) of Multivariate Polynomial Ring in x_0, x_1, x_2, x_3, x_4, x_5, x_6 over Rational Field,
 Ideal (x_2, x_3) of Multivariate Polynomial Ring in x_0, x_1, x_2, x_3, x_4, x_5, x_6 over Rational Field,
 Ideal (x_1, x_4, x_6) of Multivariate Polynomial Ring in x_0, x_1, x_2, x_3, x_4, x_5, x_6 over Rational Field]

In [122]:
fixed_loci

[Ideal (x_5, x_0) of Multivariate Polynomial Ring in x_0, x_1, x_2, x_3, x_4, x_5, x_6 over Rational Field,
 Ideal (x_1*x_3 - x_2*x_4) of Multivariate Polynomial Ring in x_0, x_1, x_2, x_3, x_4, x_5, x_6 over Rational Field]

In [123]:
gens = [{'OIDEAL': x.gens(), 'ODIM': get_odim(x)} for x in fixed_loci if x not in srideal]
gens

[{'OIDEAL': [x_1*x_3 - x_2*x_4], 'ODIM': 7}]

In [124]:
ex['OPLANES']

[{'OIDEAL': ['x2*x4-x3*x5'], 'ODIM': 7}]

In [44]:
[[y / y.subs(dict(zip(X, X_invol))) for y in x['OIDEAL']] for x in gens]

[[1], [-1], [1, 1, 1]]

In [82]:
from sage.rings.finite_rings.integer_mod_ring import Integers
S = Integers(2, is_field=True)[tuple(f'u_{i}' for i in range(k - n))]
U = np.array(S.gens())

In [91]:
a = S.ideal()
terms = gens[1]['OIDEAL']
for t in terms:
    rhs = (1 - int(t / t.subs(dict(zip(X, X_invol))))) // 2
    a += S.ideal((t.exponents()[0] @ rescws) @ U - rhs)

In [92]:
a.groebner_basis()

[u_1 + 1]

In [23]:
6 ^ invol

6

In [20]:
[1, 2] @ rescws[[2, 4]], [1, 2] @ rescws[[3, 5]]

(array([2, 1, 2]), array([2, 1, 2]))

In [22]:
[1, 2] @ rescws[[2, 4]], [1, 2] @ rescws[[3, 5]], rescws[1], rescws[0]

(array([2, 1, 2]), array([2, 1, 2]), array([1, 1, 1]), array([1, 1, 1]))

In [205]:
np.unique([x for x in (X @ rescws).prod().monomials() if x.is_squarefree()])

array([x_4*x_5*x_6, x_3*x_5*x_6, x_2*x_5*x_6, x_1*x_5*x_6, x_0*x_5*x_6,
       x_3*x_4*x_6, x_2*x_4*x_6, x_1*x_4*x_6, x_0*x_4*x_6, x_1*x_3*x_6,
       x_0*x_3*x_6, x_1*x_2*x_6, x_0*x_2*x_6, x_0*x_1*x_6, x_3*x_4*x_5,
       x_2*x_4*x_5, x_1*x_4*x_5, x_0*x_4*x_5, x_1*x_3*x_5, x_1*x_2*x_5,
       x_0*x_1*x_5, x_0*x_3*x_4, x_0*x_2*x_4, x_0*x_1*x_4, x_0*x_1*x_3,
       x_0*x_1*x_2], dtype=object)

### Volume form

In [125]:
from itertools import permutations
from sympy.combinatorics.permutations import Permutation
def complete_permutation(p, k):
    return Permutation(p + tuple(i for i in range(k) if i not in p))
vform = 0
vform_invol = 0
for p in permutations(range(k), k - n):
    vform_sign = complete_permutation(p, k).signature()
    vform_coeff = rescws[p, range(3)].prod()
    vform_monom = X[list(p)].prod()
    vform += vform_sign * vform_coeff * vform_monom
    
    p_invol = tuple(invol(i) for i in p)
    vform_invol_sign = invol.signature() * complete_permutation(p_invol, k).signature()
    vform_invol_coeff = rescws[p, range(3)].prod()
    vform_invol_monom = X[list(p_invol)].prod()
    vform_invol += vform_invol_sign * vform_invol_coeff * vform_invol_monom
vform / vform_invol

-1

In [126]:
ex['VOLFORMPARITY']

-1

In [55]:
R.ideal(a.sum() - b.sum()).minimal_associated_primes()

[Ideal (-x_0*x_4*x_5 + x_1*x_4*x_5 + 2*x_0*x_2*x_6 - 2*x_1*x_2*x_6 + 2*x_0*x_3*x_6 - 2*x_1*x_3*x_6 + 4*x_2*x_3*x_6 - x_0*x_4*x_6 + x_1*x_4*x_6 - x_0*x_5*x_6 + x_1*x_5*x_6 - 2*x_4*x_5*x_6) of Multivariate Polynomial Ring in x_0, x_1, x_2, x_3, x_4, x_5, x_6 over Rational Field]

In [30]:
a.sum() + b.sum()

8*x_0*x_3*x_6 + 8*x_1*x_3*x_6 + 6*x_0*x_4*x_6 - 4*x_1*x_4*x_6 - 4*x_2*x_4*x_6 + 4*x_0*x_5*x_6 - 6*x_1*x_5*x_6 - 4*x_2*x_5*x_6

In [28]:
srideal

[Ideal (x_0, x_5) of Multivariate Polynomial Ring in x_0, x_1, x_2, x_3, x_4, x_5, x_6 over Rational Field,
 Ideal (x_2, x_3) of Multivariate Polynomial Ring in x_0, x_1, x_2, x_3, x_4, x_5, x_6 over Rational Field,
 Ideal (x_1, x_4, x_6) of Multivariate Polynomial Ring in x_0, x_1, x_2, x_3, x_4, x_5, x_6 over Rational Field]

In [23]:
ex['VOLFORMPARITY']

0

In [214]:
Permutation((0,1,2) + tuple(i for i in range(k) if i not in (0,1,2))).signature()

1

In [216]:
rescws[(0,1,2), range(3)].prod()

0

In [213]:
a.sum() / a.sum().subs(X_sub)

(-x_0*x_1*x_2 + 2*x_0*x_1*x_3 - x_0*x_1*x_4 - x_0*x_2*x_4 + 2*x_0*x_3*x_4 + x_0*x_1*x_5 + x_1*x_2*x_5 - 2*x_1*x_3*x_5 - x_0*x_4*x_5 + x_1*x_4*x_5 + x_2*x_4*x_5 - 2*x_3*x_4*x_5 - 5*x_0*x_1*x_6 - 2*x_0*x_2*x_6 - 2*x_1*x_2*x_6 + 4*x_0*x_3*x_6 + 4*x_1*x_3*x_6 + 3*x_0*x_4*x_6 - 2*x_1*x_4*x_6 - 2*x_2*x_4*x_6 + 4*x_3*x_4*x_6 + 2*x_0*x_5*x_6 - 3*x_1*x_5*x_6 - 2*x_2*x_5*x_6 + 4*x_3*x_5*x_6 + x_4*x_5*x_6)/(-x_0*x_1*x_2 + x_0*x_1*x_3 + x_1*x_2*x_3 - x_0*x_2*x_4 - x_1*x_2*x_4 + x_0*x_3*x_4 + x_1*x_3*x_4 - x_2*x_3*x_4 + 2*x_1*x_2*x_5 - 2*x_1*x_3*x_5 + 2*x_2*x_4*x_5 - 2*x_3*x_4*x_5 - 2*x_0*x_1*x_6 - 2*x_0*x_2*x_6 - 5*x_1*x_2*x_6 - 2*x_0*x_3*x_6 - 3*x_1*x_3*x_6 + 2*x_2*x_3*x_6 - 2*x_0*x_4*x_6 - 2*x_1*x_4*x_6 + 3*x_2*x_4*x_6 + x_3*x_4*x_6 + 4*x_1*x_5*x_6 + 4*x_2*x_5*x_6 + 4*x_3*x_5*x_6 + 4*x_4*x_5*x_6)

In [210]:
np.vectorize(lambda x: x - x.subs(X_sub))(a)

array([0, -x_0*x_1*x_3 + x_1*x_2*x_5, 2*x_0*x_1*x_6 - 2*x_1*x_2*x_6, 0,
       -x_0*x_3*x_4 + x_2*x_4*x_5, 2*x_0*x_4*x_6 - 2*x_2*x_4*x_6, 0,
       2*x_0*x_3*x_6 - 2*x_2*x_5*x_6, 2*x_0*x_1*x_3 - 2*x_1*x_2*x_5, 0,
       4*x_1*x_3*x_6 - 4*x_1*x_5*x_6, 2*x_0*x_3*x_4 - 2*x_2*x_4*x_5, 0,
       4*x_3*x_4*x_6 - 4*x_4*x_5*x_6, 4*x_0*x_3*x_6 - 4*x_2*x_5*x_6, 0,
       -x_0*x_1*x_4 + x_1*x_2*x_4, -x_1*x_3*x_4 + x_1*x_4*x_5, 0,
       -2*x_0*x_4*x_6 + 2*x_2*x_4*x_6, 2*x_3*x_4*x_6 - 2*x_4*x_5*x_6,
       -x_1*x_2*x_3 + x_0*x_1*x_5, -2*x_1*x_3*x_6 + 2*x_1*x_5*x_6,
       x_2*x_3*x_4 - x_0*x_4*x_5, 2*x_3*x_4*x_6 - 2*x_4*x_5*x_6,
       -2*x_2*x_3*x_6 + 2*x_0*x_5*x_6, -5*x_0*x_1*x_6 + 5*x_1*x_2*x_6,
       5*x_1*x_3*x_6 - 5*x_1*x_5*x_6, 5*x_0*x_4*x_6 - 5*x_2*x_4*x_6,
       -5*x_3*x_4*x_6 + 5*x_4*x_5*x_6], dtype=object)

In [190]:
rescws[(4,1,2), (0,1,2)]

array([1, 1, 0])

In [193]:
rescws[2,2]

0

In [175]:
a

[((0, 1, 2), 1),
 ((0, 1, 3), -1),
 ((0, 1, 4), 1),
 ((0, 1, 5), -1),
 ((0, 1, 6), 1),
 ((0, 2, 1), -1),
 ((0, 2, 3), 1),
 ((0, 2, 4), -1),
 ((0, 2, 5), 1),
 ((0, 2, 6), -1),
 ((0, 3, 1), 1),
 ((0, 3, 2), -1),
 ((0, 3, 4), 1),
 ((0, 3, 5), -1),
 ((0, 3, 6), 1),
 ((0, 4, 1), -1),
 ((0, 4, 2), 1),
 ((0, 4, 3), -1),
 ((0, 4, 5), 1),
 ((0, 4, 6), -1),
 ((0, 5, 1), 1),
 ((0, 5, 2), -1),
 ((0, 5, 3), 1),
 ((0, 5, 4), -1),
 ((0, 5, 6), 1),
 ((0, 6, 1), -1),
 ((0, 6, 2), 1),
 ((0, 6, 3), -1),
 ((0, 6, 4), 1),
 ((0, 6, 5), -1),
 ((1, 0, 2), -1),
 ((1, 0, 3), 1),
 ((1, 0, 4), -1),
 ((1, 0, 5), 1),
 ((1, 0, 6), -1),
 ((1, 2, 0), 1),
 ((1, 2, 3), -1),
 ((1, 2, 4), 1),
 ((1, 2, 5), -1),
 ((1, 2, 6), 1),
 ((1, 3, 0), -1),
 ((1, 3, 2), 1),
 ((1, 3, 4), -1),
 ((1, 3, 5), 1),
 ((1, 3, 6), -1),
 ((1, 4, 0), 1),
 ((1, 4, 2), -1),
 ((1, 4, 3), 1),
 ((1, 4, 5), -1),
 ((1, 4, 6), 1),
 ((1, 5, 0), -1),
 ((1, 5, 2), 1),
 ((1, 5, 3), -1),
 ((1, 5, 4), 1),
 ((1, 5, 6), -1),
 ((1, 6, 0), 1),
 ((1, 6, 2), -1),
 (

In [161]:
b

array([0, 1, 3, 3, 4, 5, 6])

In [247]:
a = np.array(list(product(*(X @ rescws))))

In [265]:
Permutation([3,1,2, 0])

Permutation(0, 3)

In [263]:
X @ rescws

array([x_0 + x_1 + 2*x_3 + x_4 + x_6, x_0 + x_1 + x_2 + x_3 + 2*x_6,
       x_0 + x_1 + 2*x_2 + x_5 + x_6], dtype=object)

In [251]:
a = np.stack([x for x in a if x[:, 1].prod().is_squarefree()])

In [133]:

a = sum(a.monomial_coefficient(x) * x for x in a.monomials() if x.is_squarefree())

b = (X_invol @ rescws).prod()
b = sum(b.monomial_coefficient(x) * x for x in b.monomials() if x.is_squarefree())

In [266]:
from sympy.combinatorics.generators import cyclic

In [269]:
[x.array_form[k - n:] for x in cyclic(k)]

[[3, 4, 5, 6],
 [4, 5, 6, 0],
 [5, 6, 0, 1],
 [6, 0, 1, 2],
 [0, 1, 2, 3],
 [1, 2, 3, 4],
 [2, 3, 4, 5]]

In [164]:
X @ rescws

array([x_0 + x_1 + 2*x_3 + x_4 + x_6, x_0 + x_1 + x_2 + x_3 + 2*x_6,
       x_0 + x_1 + 2*x_2 + x_5 + x_6], dtype=object)

In [234]:
Permutation([[0,1],[2,3]])

Permutation(0, 1)(2, 3)

In [232]:
from itertools import permutations
a = list(permutations(range(k), k - n))

In [175]:
from itertools import product
a = np.array(list(product(*(X @ rescws))))

In [235]:
from sage.modules.free_module import FreeModule
A = FreeModule(R, k)

In [ ]:
A.

In [184]:
a.shape

(125, 3, 2)

In [185]:
a[:, :, 1].prod(axis=1)#.is_squarefree()

array([x_0^3, x_0^2*x_1, x_0^2*x_2, x_0^2*x_5, x_0^2*x_6, x_0^2*x_1,
       x_0*x_1^2, x_0*x_1*x_2, x_0*x_1*x_5, x_0*x_1*x_6, x_0^2*x_2,
       x_0*x_1*x_2, x_0*x_2^2, x_0*x_2*x_5, x_0*x_2*x_6, x_0^2*x_3,
       x_0*x_1*x_3, x_0*x_2*x_3, x_0*x_3*x_5, x_0*x_3*x_6, x_0^2*x_6,
       x_0*x_1*x_6, x_0*x_2*x_6, x_0*x_5*x_6, x_0*x_6^2, x_0^2*x_1,
       x_0*x_1^2, x_0*x_1*x_2, x_0*x_1*x_5, x_0*x_1*x_6, x_0*x_1^2, x_1^3,
       x_1^2*x_2, x_1^2*x_5, x_1^2*x_6, x_0*x_1*x_2, x_1^2*x_2, x_1*x_2^2,
       x_1*x_2*x_5, x_1*x_2*x_6, x_0*x_1*x_3, x_1^2*x_3, x_1*x_2*x_3,
       x_1*x_3*x_5, x_1*x_3*x_6, x_0*x_1*x_6, x_1^2*x_6, x_1*x_2*x_6,
       x_1*x_5*x_6, x_1*x_6^2, x_0^2*x_3, x_0*x_1*x_3, x_0*x_2*x_3,
       x_0*x_3*x_5, x_0*x_3*x_6, x_0*x_1*x_3, x_1^2*x_3, x_1*x_2*x_3,
       x_1*x_3*x_5, x_1*x_3*x_6, x_0*x_2*x_3, x_1*x_2*x_3, x_2^2*x_3,
       x_2*x_3*x_5, x_2*x_3*x_6, x_0*x_3^2, x_1*x_3^2, x_2*x_3^2,
       x_3^2*x_5, x_3^2*x_6, x_0*x_3*x_6, x_1*x_3*x_6, x_2*x_3*x_6,
       x_3*x_5*x_6, x_3*x

In [142]:
q = (X @ rescws).prod()
q_monoms = [x for x in q.monomials() if x.is_squarefree()]

In [159]:
Permutationnp.arange(k) * (1 - np.array(q_monoms[0].exponents())))

ValueError: All elements must be unique in a cycle.

In [302]:
def cycle(arr, orig = None):
    if arr == orig:
        return []
    if not orig:
        orig = arr
    new_arr = arr[:]
    new_arr.append(new_arr.pop(0))
    return [arr] + cycle(new_arr, orig = orig)

def is_same_parity(perm0, perm1):
    """Check if 2 permutations are of equal parity.

    Assume that both permutation lists are of equal length
    and have the same elements. No need to check for these
    conditions.
    """
    perm1 = perm1[:] ## copy this list so we don't mutate the original

    transCount = 0
    for loc in range(len(perm0) - 1):                         # Do (len - 1) transpositions
        p0 = perm0[loc]
        p1 = perm1[loc]
        if p0 != p1:
            sloc = perm1[loc:].index(p0) + loc          # Find position in perm1
            perm1[loc], perm1[sloc] = p0, p1          # Swap in perm1
            transCount += 1

    # Even number of transpositions means equal parity
    if transCount % 2 == 0:
        return True
    else:
        return False

def volform_parity(ndivs, invol):
    order = list(range(ndivs))
    for x, y in invol:
        temp = order[x]
        order[x] = order[y]
        order[y] = temp
    perm = Permutation(order)
    cycles = cycle(list(range(ndivs)))
    volform_term_parities = [is_same_parity(cycles[i][1:], [i^perm for i in cycles[i^perm][1:]]) for i in range(ndivs)]
    if all(x for x in volform_term_parities):
        return 1
    elif all(not x for x in volform_term_parities):
        return -1
    else:
        return 0

In [498]:
invol

Permutation(6)(2, 3)(4, 5)

In [519]:
rescws

array([[1, 1, 1],
       [1, 1, 1],
       [0, 1, 2],
       [2, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [1, 2, 1]])

In [516]:
((X @ rescws) / (X_invol @ rescws))[0].reduce(lin_ideal)

TypeError: reduce() takes no arguments (1 given)

In [512]:
lin_ideal = R.ideal(*(X @ dresverts))

In [514]:
[x.reduce() for x in (X @ rescws) / (X_invol @ rescws)]

[None, None, None]

In [503]:
a = R.ideal(list((X  X_invol) @ rescws))

In [504]:
a.minimal_associated_primes()

[Ideal (x_4 + x_5 - 2*x_6, x_0 + x_1 + x_2 + x_3 + 1/2*x_4 + 1/2*x_5 + x_6) of Multivariate Polynomial Ring in x_0, x_1, x_2, x_3, x_4, x_5, x_6 over Rational Field]

In [482]:
invol.signature()

1

In [484]:
rescws[invol.array_form].prod(axis=1)

array([1, 1, 0, 0, 0, 0, 2])

In [303]:
cycle([1,2,3])

[[1, 2, 3], [2, 3, 1], [3, 1, 2]]

In [493]:
import itertools
from collections import Counter
a = Counter(itertools.product(*rescws.T))

In [495]:
rescws

array([[1, 1, 1],
       [1, 1, 1],
       [0, 1, 2],
       [2, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [1, 2, 1]])

In [494]:
a

Counter({(1, 1, 1): 64,
         (1, 1, 2): 16,
         (1, 1, 0): 32,
         (1, 0, 1): 32,
         (1, 0, 2): 8,
         (1, 0, 0): 16,
         (1, 2, 1): 16,
         (1, 2, 2): 4,
         (1, 2, 0): 8,
         (0, 1, 1): 32,
         (0, 1, 2): 8,
         (0, 1, 0): 16,
         (0, 0, 1): 16,
         (0, 0, 2): 4,
         (0, 0, 0): 8,
         (0, 2, 1): 8,
         (0, 2, 2): 2,
         (0, 2, 0): 4,
         (2, 1, 1): 16,
         (2, 1, 2): 4,
         (2, 1, 0): 8,
         (2, 0, 1): 8,
         (2, 0, 2): 2,
         (2, 0, 0): 4,
         (2, 2, 1): 4,
         (2, 2, 2): 1,
         (2, 2, 0): 2})

In [461]:
[[rescws[i, j] for j in range(k - n) if rescws[i, j] > 0] for i in range(k)]

[[1, 1, 1], [1, 1, 1], [1, 2], [2, 1], [1], [1], [1, 2, 1]]

In [456]:
[[rescws[i][j] for i in range(k)] for j in range(k - n)]

[[1, 1, 0, 2, 1, 0, 1], [1, 1, 1, 1, 0, 0, 2], [1, 1, 2, 0, 0, 1, 1]]

In [436]:
a = (X @ rescws).prod()

In [451]:
b = [tuple(i for i in range(k) if m.exponents()[0][i] == 0) for m in a.monomials() if m.is_squarefree()]

In [453]:
len(b), len(set(b))

(35, 35)

In [452]:
set(b)

{(0, 1, 2, 3),
 (0, 1, 2, 4),
 (0, 1, 2, 5),
 (0, 1, 2, 6),
 (0, 1, 3, 4),
 (0, 1, 3, 5),
 (0, 1, 3, 6),
 (0, 1, 4, 5),
 (0, 1, 4, 6),
 (0, 1, 5, 6),
 (0, 2, 3, 4),
 (0, 2, 3, 5),
 (0, 2, 3, 6),
 (0, 2, 4, 5),
 (0, 2, 4, 6),
 (0, 2, 5, 6),
 (0, 3, 4, 5),
 (0, 3, 4, 6),
 (0, 3, 5, 6),
 (0, 4, 5, 6),
 (1, 2, 3, 4),
 (1, 2, 3, 5),
 (1, 2, 3, 6),
 (1, 2, 4, 5),
 (1, 2, 4, 6),
 (1, 2, 5, 6),
 (1, 3, 4, 5),
 (1, 3, 4, 6),
 (1, 3, 5, 6),
 (1, 4, 5, 6),
 (2, 3, 4, 5),
 (2, 3, 4, 6),
 (2, 3, 5, 6),
 (2, 4, 5, 6),
 (3, 4, 5, 6)}

In [442]:
b = a.monomials()[0]

In [439]:
b.is_squarefree()

False

In [315]:
np.power(-1, range(k)) * X

array([x_0, -x_1, x_2, -x_3, x_4, -x_5, x_6], dtype=object)

In [316]:
np.power(-1, range(k)) * X_invol

array([x_0, -x_1, x_3, -x_2, x_5, -x_4, x_6], dtype=object)

In [310]:
(np.power(-1, range(k)) * X) @ rescws 

array([x_0 - x_1 - 2*x_3 + x_4 + x_6, x_0 - x_1 + x_2 - x_3 + 2*x_6,
       x_0 - x_1 + 2*x_2 - x_5 + x_6], dtype=object)

In [314]:
((np.power(-1, range(k)) * X_invol) @ rescws).prod() / ((np.power(-1, range(k)) * X) @ rescws).prod()

(x_0^3 - 3*x_0^2*x_1 + 3*x_0*x_1^2 - x_1^3 - 3*x_0^2*x_2 + 6*x_0*x_1*x_2 - 3*x_1^2*x_2 + 2*x_0*x_2^2 - 2*x_1*x_2^2 + 3*x_0^2*x_3 - 6*x_0*x_1*x_3 + 3*x_1^2*x_3 - 8*x_0*x_2*x_3 + 8*x_1*x_2*x_3 + 4*x_2^2*x_3 + 2*x_0*x_3^2 - 2*x_1*x_3^2 - 4*x_2*x_3^2 - x_0^2*x_4 + 2*x_0*x_1*x_4 - x_1^2*x_4 + 3*x_0*x_2*x_4 - 3*x_1*x_2*x_4 - 2*x_2^2*x_4 - x_0*x_3*x_4 + x_1*x_3*x_4 + 2*x_2*x_3*x_4 + x_0^2*x_5 - 2*x_0*x_1*x_5 + x_1^2*x_5 - x_0*x_2*x_5 + x_1*x_2*x_5 + 3*x_0*x_3*x_5 - 3*x_1*x_3*x_5 - 2*x_2*x_3*x_5 + 2*x_3^2*x_5 - x_0*x_4*x_5 + x_1*x_4*x_5 + x_2*x_4*x_5 - x_3*x_4*x_5 + 4*x_0^2*x_6 - 8*x_0*x_1*x_6 + 4*x_1^2*x_6 - 8*x_0*x_2*x_6 + 8*x_1*x_2*x_6 + 2*x_2^2*x_6 + 8*x_0*x_3*x_6 - 8*x_1*x_3*x_6 - 12*x_2*x_3*x_6 + 2*x_3^2*x_6 - 3*x_0*x_4*x_6 + 3*x_1*x_4*x_6 + 5*x_2*x_4*x_6 - x_3*x_4*x_6 + 3*x_0*x_5*x_6 - 3*x_1*x_5*x_6 - x_2*x_5*x_6 + 5*x_3*x_5*x_6 - 2*x_4*x_5*x_6 + 5*x_0*x_6^2 - 5*x_1*x_6^2 - 5*x_2*x_6^2 + 5*x_3*x_6^2 - 2*x_4*x_6^2 + 2*x_5*x_6^2 + 2*x_6^3)/(x_0^3 - 3*x_0^2*x_1 + 3*x_0*x_1^2 - x_1^3 + 3*x_

In [291]:
V = (X @ rescws).prod()
V_invol = invol.signature() * (X_invol @ rescws).prod()
V_invol / V

1

In [301]:
invol

Permutation(6)(2, 3)(4, 5)

In [298]:
X @ rescws.prod(axis=1)

x_0 + x_1 + 2*x_6

In [300]:
X_invol @ rescws.prod(axis=1)

x_0 + x_1 + 2*x_6

In [268]:
from sympy.combinatorics import Permutation

In [295]:
rescws

array([[1, 1, 1],
       [1, 1, 1],
       [0, 1, 2],
       [2, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [1, 2, 1]])

In [293]:
X @ rescws

array([x_0 + x_1 + 2*x_3 + x_4 + x_6, x_0 + x_1 + x_2 + x_3 + 2*x_6,
       x_0 + x_1 + 2*x_2 + x_5 + x_6], dtype=object)

In [294]:
X_invol @ rescws

array([x_0 + x_1 + 2*x_2 + x_5 + x_6, x_0 + x_1 + x_2 + x_3 + 2*x_6,
       x_0 + x_1 + 2*x_3 + x_4 + x_6], dtype=object)

In [292]:
(X - X_invol) @ rescws

array([-2*x_2 + 2*x_3 + x_4 - x_5, 0, 2*x_2 - 2*x_3 - x_4 + x_5],
      dtype=object)

In [248]:
ex['VOLFORMPARITY']

-1

### Kahler form
The even parity requirement just restricts the coefficients of the Kahler form

In [155]:
[(x - y).reduce(lin_ideal) for x, y in zip(X, X_invol)]

[2*x_4 + 4*x_5 - x_6,
 2*x_4 + 4*x_5 - x_6,
 -2*x_4 - 4*x_5 + x_6,
 -2*x_4 - 4*x_5 + x_6,
 -2*x_4 - 4*x_5 + x_6,
 2*x_4 + 4*x_5 - x_6,
 0]

In [156]:
Rbar = R.quotient(lin_ideal)

In [157]:
simplex_vols = np.array([abs(matrix(ZZ, dresverts[x]).det()) for x in triang])
simplex_vols = simplex_vols / simplex_vols.min()
basis = [X[i] for i in range(k) if i not in triang[simplex_vols.argmin()]]

In [414]:
RR = QQ[tuple([f'x_{i}' for i in range(k)] + [f'a_{i}' for i in range(k)])]
XX = np.array(RR.gens()[:k])
XX_invol = XX[invol.array_form]
AA = np.array(RR.gens()[k:])

In [415]:
lin_ideal = RR.ideal(*(RR.gens()[:k] @ dresverts))
[x.reduce(lin_ideal) for x in XX]

[1/2*x_4 + 1/2*x_5 + 1/2*x_6,
 1/2*x_4 + 1/2*x_5 + 1/2*x_6,
 -1/2*x_4 + 3/2*x_5 + 1/2*x_6,
 3/2*x_4 - 1/2*x_5 + 1/2*x_6,
 x_4,
 x_5,
 x_6]

In [416]:
qq = RR.ideal((AA @ (XX - XX_invol)).reduce(lin_ideal))

In [410]:
qq

Ideal (-2*x_4*a_2 + 2*x_5*a_2 + 2*x_4*a_3 - 2*x_5*a_3 + x_4*a_4 - x_5*a_4 - x_4*a_5 + x_5*a_5) of Multivariate Polynomial Ring in x_0, x_1, x_2, x_3, x_4, x_5, x_6, a_0, a_1, a_2, a_3, a_4, a_5, a_6 over Rational Field

In [411]:
qq.minimal_associated_primes()

[Ideal (2*a_2 - 2*a_3 - a_4 + a_5) of Multivariate Polynomial Ring in x_0, x_1, x_2, x_3, x_4, x_5, x_6, a_0, a_1, a_2, a_3, a_4, a_5, a_6 over Rational Field,
 Ideal (x_4 - x_5) of Multivariate Polynomial Ring in x_0, x_1, x_2, x_3, x_4, x_5, x_6, a_0, a_1, a_2, a_3, a_4, a_5, a_6 over Rational Field]

In [378]:
lin_ideal = RR.ideal(*(RR.gens()[:k] @ dresverts))
lin_ideal_invol = RR.ideal(*([RR.gen(i) for i in invol.array_form] @ dresverts))
a = [x.reduce(lin_ideal) - y.reduce(lin_ideal_invol) for x, y in zip(RR.gens()[:k], [RR.gen(i) for i in invol.array_form])]
a = [RR.gen(k + i) * a[i] for i in range(k)]

In [379]:
a

[0,
 0,
 -2*x_4*a_2 + 2*x_5*a_2,
 2*x_4*a_3 - 2*x_5*a_3,
 x_4*a_4 - x_5*a_4,
 -x_4*a_5 + x_5*a_5,
 0]

In [224]:
lin_ideal = R.ideal(*(X @ dresverts))
# lin_ideal_invol = R.ideal(*(X_invol @ dresverts))
invol_ideal = R.ideal(*[X - X_invol])
[(x - y).reduce(lin_ideal + invol_ideal) - y.reduce(lin_ideal_invol + invol_ideal) for x, y in zip(X, X_invol)]

[0, 0, -2*x_4 + 2*x_5, 2*x_4 - 2*x_5, x_4 - x_5, -x_4 + x_5, 0]

In [226]:
lin_ideal.minimal_associated_primes()

[Ideal (x_3 - 3/2*x_4 + 1/2*x_5 - 1/2*x_6, x_2 + x_3 - x_4 - x_5 - x_6, x_1 - x_3 + x_4 - x_5, x_0 + x_1 + x_2 + x_3 - 2*x_4 - 2*x_5 - 2*x_6) of Multivariate Polynomial Ring in x_0, x_1, x_2, x_3, x_4, x_5, x_6 over Rational Field]

In [253]:
a = Xbar[0]

In [259]:
lin_ideal = R.ideal(*(X @ dresverts))
Xbar = np.array([x.reduce(lin_ideal) for x in X])
[[y.coefficient(x) for x in X] for y in Xbar]

[[0, 0, 0, 0, 1/2, 1/2, 1/2],
 [0, 0, 0, 0, 1/2, 1/2, 1/2],
 [0, 0, 0, 0, -1/2, 3/2, 1/2],
 [0, 0, 0, 0, 3/2, -1/2, 1/2],
 [0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 1]]

In [258]:
lin_ideal_invol = R.ideal(*(X_invol @ dresverts))
Xbar_invol = np.array([x.reduce(lin_ideal_invol) for x in X_invol])
[[y.coefficient(x) for x in X_invol] for y in Xbar_invol]

[[0, 0, 0, 0, 1/2, 1/2, 1/2],
 [0, 0, 0, 0, 1/2, 1/2, 1/2],
 [0, 0, 0, 0, -1/2, 3/2, 1/2],
 [0, 0, 0, 0, 3/2, -1/2, 1/2],
 [0, 0, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 1]]

In [261]:
Xbar

array([1/2*x_4 + 1/2*x_5 + 1/2*x_6, 1/2*x_4 + 1/2*x_5 + 1/2*x_6,
       -1/2*x_4 + 3/2*x_5 + 1/2*x_6, 3/2*x_4 - 1/2*x_5 + 1/2*x_6, x_4,
       x_5, x_6], dtype=object)

In [262]:
Xbar_invol

array([1/2*x_4 + 1/2*x_5 + 1/2*x_6, 1/2*x_4 + 1/2*x_5 + 1/2*x_6,
       3/2*x_4 - 1/2*x_5 + 1/2*x_6, -1/2*x_4 + 3/2*x_5 + 1/2*x_6, x_5,
       x_4, x_6], dtype=object)

In [264]:
Xbar + Xbar_invol

array([x_4 + x_5 + x_6, x_4 + x_5 + x_6, x_4 + x_5 + x_6, x_4 + x_5 + x_6,
       x_4 + x_5, x_4 + x_5, 2*x_6], dtype=object)

In [167]:
lin_ideal

Ideal (-x_0 - x_1 - x_2 - x_3 - x_4 - x_5 + x_6, x_3 + 2*x_4 + 3*x_5 - x_6, 2*x_1 + 3*x_2 + x_5 - x_6, -x_1 - 2*x_2 - x_4 - 2*x_5 + x_6) of Multivariate Polynomial Ring in x_0, x_1, x_2, x_3, x_4, x_5, x_6 over Rational Field

In [168]:
lin_ideal_invol

Ideal (-x_0 - x_1 - x_2 - x_3 - x_4 - x_5 + x_6, x_0 + 2*x_1 + 3*x_2 - x_6, x_2 + 2*x_4 + 3*x_5 - x_6, -x_1 - 2*x_2 - x_4 - 2*x_5 + x_6) of Multivariate Polynomial Ring in x_0, x_1, x_2, x_3, x_4, x_5, x_6 over Rational Field

In [67]:
a = [x.reduce(linideal) for x in X]
b = [x.reduce(linideal) for x in X_invol]

In [69]:
a

[x_5,
 3*x_4 + 4*x_5 - x_6,
 -2*x_4 - 3*x_5 + x_6,
 -2*x_4 - 3*x_5 + x_6,
 x_4,
 x_5,
 x_6]

In [70]:
b

[-2*x_4 - 3*x_5 + x_6,
 x_4,
 x_5,
 x_5,
 3*x_4 + 4*x_5 - x_6,
 -2*x_4 - 3*x_5 + x_6,
 x_6]

In [49]:
[(X[i]-X[3]).reduce(linideal) for i in range(k) if i in basis_triang]

[2*x_4 + 4*x_5 - x_6, 5*x_4 + 7*x_5 - 2*x_6, 0, 2*x_4 + 3*x_5]

In [119]:
[x - y for x, y in zip(Rbar.gens(), [Rbar.gen(i) for i in invol.array_form])]

[2*x_4bar + 4*x_5bar - x_6bar,
 2*x_4bar + 4*x_5bar - x_6bar,
 -2*x_4bar - 4*x_5bar + x_6bar,
 -2*x_4bar - 4*x_5bar + x_6bar,
 -2*x_4bar - 4*x_5bar + x_6bar,
 2*x_4bar + 4*x_5bar - x_6bar,
 0]

In [73]:
DtoJmat = [[1 if i==j else 0 for i in range(k)] for j in range(k) if j not in basis_triang]

In [74]:
[[JJ[j],sum([DtoJmat[j][i]*X[i] for i in range(len(DD))])] for j in range(len(JJ))]

[[0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 0, 1, 0]]

In [ ]:
R.